In [ ]:
# Ajuste de un modelo de regresion lineal multiple a los datos de peso-edad-presion

  library(alr4)
 
#  peso.edad.presion <- read.table("c:\\users\\martin\\mydocuments\\cursos\\regresion\\peso-edad-presion.dat",header=T)
#  peso.edad.presion <- read.table("c:\\documents and settings\\martin\\mis documentos\\cursos\\regresion\\peso-edasd-presion.dat",header=T)
  peso.edad.presion <- read.table("C:\\Users\\mhfel\\OneDrive\\Documentos\\Cursos\\RegresionLineal\\peso-edad-presion.dat",header=T)
  
  
  peso.edad.presion
  edad <- peso.edad.presion$Edad
  presion <- peso.edad.presion$Presion
  peso <- peso.edad.presion$Peso
  
# Graficas de dispersion  
  plot(peso,presion)
  plot(edad,presion)
  
# Matriz de graficas de dispersion  
  pairs( peso.edad.presion )
  pairs(~ presion + peso + edad)
  scatterplotMatrix( peso.edad.presion )
  scatterplotMatrix( ~ presion + peso + edad  )
  
  
# Ajuste del modelo presion = beta0 + beta1 * peso + beta2 * edad  
  
  reg.presion.peso.edad <- lm(presion ~ peso + edad) 
  summary(reg.presion.peso.edad)
  anova(reg.presion.peso.edad)
  coefficients(reg.presion.peso.edad)
  confint(reg.presion.peso.edad)
  confint(reg.presion.peso.edad, level = 0.983) # Bonferroni ajuste
  residuals(reg.presion.peso.edad)
  fitted.values(reg.presion.peso.edad)
  vcov(reg.presion.peso.edad)
  
# Prueba de hipotesis lineal general
  
# H0: beta1=beta2=0
  A.matrix <- matrix(c(0,1,0,0,0,1),nrow=2,byrow=T)
  linearHypothesis(reg.presion.peso.edad, hypothesis.matrix=A.matrix, rhs=c(0,0))
  linearHypothesis(reg.presion.peso.edad, c("peso = 0", "edad = 0"))
  linearHypothesis(reg.presion.peso.edad, c("peso", "edad"))

# Calculo de la estadistica de prueba usando diferencia de SCE
  
  reg0.presion.peso.edad <- lm(presion ~ 1)
  summary(reg0.presion.peso.edad)
  anova(reg0.presion.peso.edad)
  
  anova(reg0.presion.peso.edad,reg.presion.peso.edad)
  
  
# H0: beta1=beta2
  A.matrix <- matrix(c(0,1,-1),nrow=1,byrow=T)
  linearHypothesis(reg.presion.peso.edad, hypothesis.matrix=A.matrix, rhs=c(0))
  linearHypothesis(reg.presion.peso.edad, c("peso = edad"))
  
# Calculo de la estadistica de prueba usando diferencia de SCE
  
  peso.edad <- peso + edad
  reg0.presion.peso.edad <- lm(presion ~ peso.edad )
  summary(reg0.presion.peso.edad)
  anova(reg0.presion.peso.edad)
  
  
  anova(reg0.presion.peso.edad,reg.presion.peso.edad)
  
  
# Prueba de hipotesis acerca de la inclusion de terminos cuadraticos
  peso2 <- peso^2
  edad2 <- edad^2
  pesoedad <- peso * edad
  pesoedadpresion2 <- lm( presion ~ peso + edad + peso2 + edad2 + pesoedad )
  summary(pesoedadpresion2)
  A.matrix <- matrix(c(0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1),nrow=3,byrow=T)
  linearHypothesis(pesoedadpresion2, hypothesis.matrix=A.matrix, rhs=c(0,0,0))
  linearHypothesis(pesoedadpresion2, c("peso2 = 0", "edad2 = 0", "pesoedad = 0"))
  linearHypothesis(pesoedadpresion2, c("peso2", "edad2", "pesoedad"))
  
# Calculo de la estadistica de prueba usando diferencia de SCE
  
  anova(reg.presion.peso.edad,pesoedadpresion2)
  
  
  
# Valores ajustados con intervalos de confianza de las medias condicionales y de observaciones futuras 

# Sin ajuste de Scheffe
  est.media <- predict(reg.presion.peso.edad,interval="confidence")
  pred.resp <- predict(reg.presion.peso.edad,interval="prediction")
  
  sigma.est <- summary(reg.presion.peso.edad)$sigma
  
# Con ajuste de Scheffe
  est.media[,3] <- est.media[,1]+sigma.est*sqrt(hatvalues(reg.presion.peso.edad))*sqrt(3*qf(.95,df1=3,df2=10))
  est.media[,2] <- est.media[,1]-sigma.est*sqrt(hatvalues(reg.presion.peso.edad))*sqrt(3*qf(.95,df1=3,df2=10))
  
  pred.resp[,3] <- pred.resp[,1]+sigma.est*sqrt(1+hatvalues(reg.presion.peso.edad))*sqrt(3*qf(.95,df1=3,df2=10))
  pred.resp[,2] <- pred.resp[,1]-sigma.est*sqrt(1+hatvalues(reg.presion.peso.edad))*sqrt(3*qf(.95,df1=3,df2=10))
  
    
# Estimacion puntual y de intervalo de medias condicionales y observaciones futuras
# en valores de la variable regresora no incluidas en los datos originales
  
# Sin ajuste Bonferroni
  pred.frame <-  data.frame(peso=c(158.5,160.9),edad=c(32,42))
  est.media <- predict(reg.presion.peso.edad,interval="confidence",newdata=pred.frame) 
  pred.resp <- predict(reg.presion.peso.edad,interval="prediction",newdata=pred.frame)

  x0.matriz <- matrix(c(1,158.5,32,1,160.9,42),nrow=2, byrow=T)
  
  
# Con ajuste Bonferroni  
  est.media[,3] <- est.media[,1]+sqrt( diag(x0.matriz %*% vcov(reg.presion.peso.edad) %*% t(x0.matriz) ) ) * qt(0.9875,df=10)
  est.media[,2] <- est.media[,1]-sqrt( diag(x0.matriz %*% vcov(reg.presion.peso.edad) %*% t(x0.matriz) ) ) * qt(0.9875,df=10)
  
  pred.resp[,3] <- pred.resp[,1]+sqrt( sigma.est^2+diag(x0.matriz %*% vcov(reg.presion.peso.edad) %*% t(x0.matriz) ) ) * qt(0.9875,df=10)
  pred.resp[,2] <- pred.resp[,1]-sqrt( sigma.est^2+diag(x0.matriz %*% vcov(reg.presion.peso.edad) %*% t(x0.matriz) ) ) * qt(0.9875,df=10)
  
# Otra opcion para realizar ajuste de Bonferroni  
  est.media <- predict(reg.presion.peso.edad,interval="confidence",newdata=pred.frame, level=0.975) 
  pred.resp <- predict(reg.presion.peso.edad,interval="prediction",newdata=pred.frame, level=0.975)
  
  
  
  
  
# Modelo centrado  
  peso.cent <- peso - mean(peso)
  edad.cent <- edad - mean(edad)
  reg.presion.peso.edad.centrada <- lm(presion ~ peso.cent + edad.cent)
  summary(reg.presion.peso.edad.centrada)
  vcov(reg.presion.peso.edad.centrada)
  
  
  
  
# Modelo sin intercepto
  peso.edad.presion.nointercepto <- lm (presion ~ peso + edad - 1)
  summary(peso.edad.presion.nointercepto)
  
  
  
  
  
  
  
  

# Graficas de variable agregada  
  regre.presion.edad <- lm(presion ~ edad)
  regre.peso.edad <- lm(peso ~ edad )
  plot( residuals(regre.peso.edad),residuals(regre.presion.edad))
  abline(lm(residuals(regre.presion.edad)~residuals(regre.peso.edad) ))
  regre.presion.peso <- (lm( presion ~ peso))
  regre.edad.peso <- lm(edad ~ peso )
  plot( residuals(regre.edad.peso),residuals(regre.presion.peso))
  abline(lm(residuals(regre.presion.peso)~residuals(regre.edad.peso) ))
  
# Alternativa usando funcion avPlots
  pesoedadpresion <- lm (presion ~ peso + edad)
  avPlots(pesoedadpresion, lwd=-1)
  avPlots(pesoedadpresion)

# Graficas de residuos  
  
  plot(fitted.values(pesoedadpresion),rstudent(pesoedadpresion))
  abline(h=0)
  plot(peso,rstudent(pesoedadpresion))
  abline(h=0)
  plot(edad,rstudent(pesoedadpresion))
  abline(h=0)
  
# Graficas de residuos y Pruebas de no linealidad
  residualPlots(pesoedadpresion, terms= ~. , type = "rstudent", quadratic=FALSE)

# Pruebas sobre homogeneidad de la varianza
  ncvTest(pesoedadpresion)
  ncvTest(pesoedadpresion, ~ peso + edad)
  ncvTest(pesoedadpresion, ~ peso)
  ncvTest(pesoedadpresion, ~ edad)
  
  
# Grafica de probabilidad normal
  qqnorm(residuals(pesoedadpresion)) 
  qqline(residuals(pesoedadpresion))
  qqnorm(rstudent(pesoedadpresion))
  qqline(rstudent(pesoedadpresion))
  
  qqPlot(rstudent(pesoedadpresion))
  qqPlot(rstudent(pesoedadpresion), envelope=FALSE)
  
# Prueba de puntos discrepantes
  
  outlierTest(pesoedadpresion)
  
  
  
  
  summary(pesoedadpresion)
  anova(pesoedadpresion)
  coefficients(pesoedadpresion)
  residuals(pesoedadpresion)
  fitted.values(pesoedadpresion)
  vcov(pesoedadpresion)
  lm.influence(pesoedadpresion)
  
  scatterplot(presion ~ peso + presion, data=peso.edad.presion, ellipse=TRUE)
  
# Valores ajustados con intervalos de confianza de las medias condicionales y de observaciones futuras 
  predict(pesoedadpresion,interval="confidence")
  predict(pesoedadpresion,interval="prediction")
  
# Estimacion puntual y de intervalo de medias condicionales y observaciones futuras
# en valores de la variable regresora no incluidas en los datos originales

  pred.frame <-  data.frame(peso=c(158.5,160.9),edad=c(32,42))
  predict(pesoedadpresion,interval="prediction",newdata=pred.frame)
  predict(pesoedadpresion,interval="confidence",newdata=pred.frame) 


  presionpeso <- lm( presion ~ peso )
  edadpeso <- lm( edad ~ peso )
  plot( residuals(edadpeso), residuals(presionpeso) )
  resid12 <- lm( residuals(presionpeso) ~ residuals(edadpeso) )
  summary(resid12)


  
 
  
  
# Prueba de puntos discrepantes
  
  outlierTest(pesoedadpresion)
  
  
# Diagnosticos de influencia
  
  infIndexPlot(pesoedadpresion)
  
  influencePlot(pesoedadpresion)
  
  cooks.distance(pesoedadpresion)
  
  pf(cooks.distance(pesoedadpresion), 3, 10)
  


# Hipotesis lineal general  
  
  X <- model.matrix(pesoedadpresion)
  A <- matrix(c(0,1,-0,0,0,1),nrow=2,ncol=3,byrow=T)
  g <- matrix(c(0,0),nrow=2,ncol=1,byrow=T)
  beta <- matrix(coefficients(pesoedadpresion),nrow=3,ncol=1,byrow=T)
  F0 <- t(A %*% beta - g)%*%solve(A %*%solve(t(X)%*%X)%*%t(A))%*%(A %*% beta - g) / (2*2.509^2)
  F0 <- F0[1,1]
  1 -pf(F0,2,10)

#Otra manera de realizar la prueba de Hipotesis  lineal general  
  pesoedadpresion0 <- lm(presion ~ 1)
  anova(pesoedadpresion,pesoedadpresion0)
  

  
  